In [ ]:
from minisom import MiniSom
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import RegularPolygon
from matplotlib import cm

from bokeh.colors import RGB
from bokeh.models import ColumnDataSource, Plot, Hex, Dot, HoverTool
from bokeh.io import curdoc, show, output_notebook

output_notebook()

In [ ]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt', 
                   names=['area', 'perimeter', 'compactness', 'length_kernel', 'width_kernel',
                          'asymmetry_coefficient', 'length_kernel_groove', 'target'], 
                   sep='\t+')

data = data[data.columns[:-1]]
# data normalization
data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
data = data.values

# Initialization and training
som = MiniSom(15, 15, data.shape[1], sigma=1.5, learning_rate=.7, activation_distance='euclidean',
              topology='hexagonal', neighborhood_function='gaussian', random_seed=10)

som.train(data, 1000, verbose=True)

In [ ]:
f = plt.figure(figsize=(10,10))
ax = f.add_subplot(111)

ax.set_aspect('equal')

xx, yy = som.get_euclidean_coordinates()
umatrix = som.distance_map()
weights = som.get_weights()

for i in range(weights.shape[0]):
    for j in range(weights.shape[1]):
        wy = yy[(i, j)] * 2 / np.sqrt(3) * 3 / 4
        hex = RegularPolygon((xx[(i, j)], wy), 
                             numVertices=6, 
                             radius=.95 / np.sqrt(3),
                             facecolor=cm.Blues(umatrix[i, j]), 
                             alpha=.4, 
                             edgecolor='gray')
        ax.add_patch(hex)
for x in data:
    w = som.winner(x) 
    # place a marker on the winning position for the sample xx
    wx, wy = som.convert_map_to_euclidean(w) 
    wy = wy * 2 / np.sqrt(3) * 3 / 4
    plt.plot(wx, wy, 
             markerfacecolor='None',
             markeredgecolor='black', 
             markersize=12, 
             markeredgewidth=2)

plt.show()

***

## HexagonalTopology: bokeh

In [ ]:
tile_centres_column = []
tile_centres_row = []
hex_colour = []
for i in range(weights.shape[0]):
    for j in range(weights.shape[1]):
        wy = yy[(i, j)] * 2 / np.sqrt(3) * 3 / 4
        tile_centres_column.append(xx[(i, j)])
        tile_centres_row.append(wy)
        hex_colour.append(cm.Blues(umatrix[i, j]))
        
weight_x = []
weight_y = []
for x in data:
    w = som.winner(x)
    wx, wy = som.convert_map_to_euclidean(xy=w)
    wy = wy * 2 / np.sqrt(3) * 3 / 4
    weight_x.append(wx)
    weight_y.append(wy)
    
# convert matplotlib colour palette to bokeh colour palette
blues_plt_hex = [(255 * np.array(i)).astype(int) for i in hex_colour]
blues_bokeh_hex = [RGB(*tuple(rgb)).to_hex() for rgb in blues_plt_hex]

In [ ]:
plot = Plot(title=None, 
            plot_width=800, plot_height=800, 
            match_aspect=True,
            toolbar_location='right')

source_hex = ColumnDataSource(dict(
    x=tile_centres_column,
    y=tile_centres_row,
    c=blues_bokeh_hex
))

source_dot = ColumnDataSource(dict(
    wx=weight_x,
    wy=weight_y
))

hex = Hex(y='x', x='y',
          size=100*(.95 / np.sqrt(3)),
          fill_color='c')
plot.add_glyph(source_or_glyph=source_hex, glyph=hex)

dot = Dot(y='wx', x='wy', size=30, line_color='black')
plot.add_glyph(source_or_glyph=source_dot, glyph=dot)

curdoc().add_root(plot)
show(plot)

In [ ]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, output_file, show, ColumnDataSource

output_file("example_stackoverflow_bokeh.html")

fig = figure(title="SOM: Hexagonal Topology",
             plot_height=800, plot_width=800,
             match_aspect=True,
             tools="wheel_zoom,save,reset")

source_hex = ColumnDataSource(
    data = dict(
        x=tile_centres_column,
        y=tile_centres_row,
        c=blues_bokeh_hex
    )
)

source_dot = ColumnDataSource(
    data=dict(
        wx=weight_x,
        wy=weight_y
    )
)

fig.hex(x='y', y='x', source=source_hex,
        size=100 * (.95 / np.sqrt(3)),
        alpha=.4,
        line_color='gray',
        fill_color='c')

fig.dot(x=weight_y, y=weight_x, 
        size=30, 
        line_color='black')

fig.add_tools(HoverTool(
    tooltips=[
        ("hex colour", '$color[hex, swatch]:c'), 
        ("(x,y)", "($x, $y)")],
    mode="mouse", 
    point_policy="follow_mouse"
))

show(fig)